In [ ]:
#SERVER PROGRAM
import os
import socket 
import threading
os.chdir('D:\\Shareable Folder')
HEADER = 64
PORT = 5050
SERVER = socket.gethostbyname(socket.gethostname())
ADDR = (SERVER, PORT)
FORMAT = 'utf-8'
DISCONNECT_MESSAGE = "!DISCONNECT"
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(ADDR)

def handle_client(conn, addr):
    print(f"[NEW CONNECTION] {addr} connected.")
    connected = True
    while connected:
        msg_length = conn.recv(HEADER).decode(FORMAT)
        if msg_length:
            msg_length = int(msg_length)
            msg = conn.recv(msg_length).decode(FORMAT)
            if msg == DISCONNECT_MESSAGE:
                break
            msg,feat = msg.split(',')
            if feat == "1":
                conn.send(f"{msg},{os.path.getsize(msg)}".encode(FORMAT))
            elif feat == "0":
                sendfile(msg,conn)
    print(f"[DISCONNECTED] {addr}")
    conn.close()

def chunks(msg,offset):
    tempfile_path = 'TEMP\\temp'
    size = os.path.getsize(msg)
    with open(msg,'rb') as file:
        with open(tempfile_path,'wb') as chunk:
            file.seek(offset)
            data = file.read(2048000)
            if not data:
                return "EOF"
            chunk.write(data)
    return tempfile_path
            
def sendfile(msg,conn):
    size = os.path.getsize(msg)
    size2 = size
    size_sent = 0
    remaining = 0
    offset = 0
    tempfile = ''
    while True:
        tempfile = chunks(msg,offset)
        if tempfile == "EOF":
            break
        with open(tempfile,'rb') as file:
            data = file.read()
            #while True:
            conn.send(data)
            print(len(data))
                #size_sent = acknowledgement(conn)
#                 if size_sent = os.path.getsize(tempfile):
            offset += 2048000
#                     break
#         while(size2 >0):
#             remaining = min(1024,size2)
#             data = file.read(remaining)
#             conn.send(data)
#             size_sent = acknowledgement(conn)
#             size2 -= size_sent
    conn.send(b'')        
    print("Sending complete")
    
def start():
    server.listen()
    print(f"[LISTENING] Server is listening on {SERVER}")
    while True:
        conn, addr = server.accept()
        
        thread = threading.Thread(target=handle_client, args=(conn, addr))
        thread.start()
        print(f"[ACTIVE CONNECTIONS] {threading.activeCount() - 1}")
print("[STARTING] server is starting...")

start()